In [1]:
def dynamic_learning_rate(step):
    print("#", step)
    total_steps = 100_000
    init_lr = 0.001
    return init_lr * (1 - step / total_steps)

In [ ]:
# Experiment 1
LR = 0.0003
GAMMA = 0.99
N_STEPS = 256
GAE_LAMBDA = 0.95

In [ ]:
# Experiment 2
LR = 0.0005
GAMMA = 0.99
N_STEPS = 256
GAE_LAMBDA = 0.95

In [2]:
# Experiment 3
LR = dynamic_learning_rate
GAMMA = 0.995
N_STEPS = 256
GAE_LAMBDA = 0.97

In [3]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.monitor import Monitor
import numpy as np


# Create the CarRacing-v2 environment
env = gym.make('CarRacing-v2')
# env = Monitor(env)  # To log episode rewards

# Wrap the environment
# env = DummyVecEnv([lambda: env])

# Create the PPO model
model = PPO('CnnPolicy',
            env,
            verbose=1,
            n_steps=N_STEPS,
            learning_rate=LR,
            gamma=GAMMA,
            gae_lambda=GAE_LAMBDA,
            tensorboard_log="./ppo_car_racing_tensorboard/",
            device='auto'
            )

# Train the model with the custom callback
model.learn(total_timesteps=50_000, tb_log_name="Experiment_1", progress_bar=True, log_interval=1)

# Save the model
model.save("ppo_car_racing_v2")

# To use the model later, load it with:
# model = PPO.load("ppo_car_racing_v2")


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
TEST2
Logging to ./ppo_car_racing_tensorboard/Experiment_1_5


Output()

----------------------------
| time/              |     |
|    fps             | 57  |
|    iterations      | 1   |
|    time_elapsed    | 4   |
|    total_timesteps | 256 |
----------------------------


-----------------------------------------
| time/                   |             |
|    fps                  | 45          |
|    iterations           | 2           |
|    time_elapsed         | 11          |
|    total_timesteps      | 512         |
| train/                  |             |
|    approx_kl            | 0.005727549 |
|    clip_fraction        | 0.141       |
|    clip_range           | 0.2         |
|    entropy_loss         | -4.24       |
|    explained_variance   | -0.0326     |
|    learning_rate        | 0.001       |
|    loss                 | 0.276       |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0108     |
|    std                  | 0.989       |
|    value_loss           | 2.06        |
-----------------------------------------


-----------------------------------------
| time/                   |             |
|    fps                  | 42          |
|    iterations           | 3           |
|    time_elapsed         | 17          |
|    total_timesteps      | 768         |
| train/                  |             |
|    approx_kl            | 0.007483428 |
|    clip_fraction        | 0.141       |
|    clip_range           | 0.2         |
|    entropy_loss         | -4.22       |
|    explained_variance   | 0.00344     |
|    learning_rate        | 0.001       |
|    loss                 | 0.555       |
|    n_updates            | 20          |
|    policy_gradient_loss | -0.00325    |
|    std                  | 0.986       |
|    value_loss           | 1.3         |
-----------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1e+03        |
|    ep_rew_mean          | -51.7        |
| time/                   |              |
|    fps                  | 41           |
|    iterations           | 4            |
|    time_elapsed         | 24           |
|    total_timesteps      | 1024         |
| train/                  |              |
|    approx_kl            | 0.0156058865 |
|    clip_fraction        | 0.214        |
|    clip_range           | 0.2          |
|    entropy_loss         | -4.21        |
|    explained_variance   | 0.0235       |
|    learning_rate        | 0.001        |
|    loss                 | 0.761        |
|    n_updates            | 30           |
|    policy_gradient_loss | -0.00329     |
|    std                  | 0.985        |
|    value_loss           | 1.32         |
------------------------------------------


----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1e+03      |
|    ep_rew_mean          | -51.7      |
| time/                   |            |
|    fps                  | 40         |
|    iterations           | 5          |
|    time_elapsed         | 31         |
|    total_timesteps      | 1280       |
| train/                  |            |
|    approx_kl            | 0.01272986 |
|    clip_fraction        | 0.169      |
|    clip_range           | 0.2        |
|    entropy_loss         | -4.21      |
|    explained_variance   | 0.0384     |
|    learning_rate        | 0.001      |
|    loss                 | 0.166      |
|    n_updates            | 40         |
|    policy_gradient_loss | -0.0069    |
|    std                  | 0.984      |
|    value_loss           | 1.09       |
----------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1e+03       |
|    ep_rew_mean          | -51.7       |
| time/                   |             |
|    fps                  | 40          |
|    iterations           | 6           |
|    time_elapsed         | 38          |
|    total_timesteps      | 1536        |
| train/                  |             |
|    approx_kl            | 0.014854586 |
|    clip_fraction        | 0.137       |
|    clip_range           | 0.2         |
|    entropy_loss         | -4.21       |
|    explained_variance   | 0.0595      |
|    learning_rate        | 0.001       |
|    loss                 | 0.37        |
|    n_updates            | 50          |
|    policy_gradient_loss | 0.00411     |
|    std                  | 0.987       |
|    value_loss           | 1.16        |
-----------------------------------------


KeyboardInterrupt: 

In [ ]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.monitor import Monitor
import numpy as np

# load model

env = gym.make('CarRacing-v2', render_mode='human')
model = PPO.load("ppo_car_racing_v2")

obs, info = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info, _ = env.step(action)
    env.render()    